In [1]:
import json
import os
import sys

## Goal
#### In this lab we will take a step-by-step aprpoach to build an agent
* Learning Objective 1 - Build a local strands agent and run it locally
* Learning Objective 2 - Get Agent core ready for AgentCore
* Learning Objective 3 - Deploy using starter toolkit and DIY Docker image
* Learning Objective 4 - Invoke agent using toolkit, boto3, and HTTPS
* Learning Objectvie 5 - Enable tools to access protected resources - API Key and OAuth Tokens
* Learning Objectvie 6 - Use OAuth and Cognito to authenticate/authorize before accessing Agent 

<img src="images/goal.jpg" width="75%">

## Learning Objective 1 - Basic agent running locally

#### As a first step of the learning journey, we will build a simple Strands Agents
* No authentication - available to all users on the account with required IAM privileges
* get_weather() is not accessing any resources secured by an API Key or OAuth 2.0 Access Tokens
* Run locally in Jupyter Notebook
  
<img src="images/Learning 1.jpg" width="500">

In [2]:
%%writefile weather_agent.py
import argparse, json
from strands import Agent, tool

@tool
def weather(city: str):
    """
    Get weather for a given city
    Params:
        city: Name of the city you want the weather for
    """
    return f"Weather in {city} is warm and sunny"

agent = Agent(tools=[weather])

def strands_agent_bedrock(payload):
    prompt = payload.get("prompt","hello")
    response = agent(prompt)
    return response

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_bedrock(json.loads(args.payload))
    print(response)

Overwriting weather_agent.py


In [3]:
!python3 weather_agent.py '{"prompt": "What is the weather now in Irvine?"}'

I'll check the current weather in Irvine for you.
Tool #1: weather
The current weather in Irvine is warm and sunny. It's a nice day out there!The current weather in Irvine is warm and sunny. It's a nice day out there!



## Learning Objective 2: Get agent code ready for AgentCore

#### 1. Make code changes
Let's now deploy our agents to AgentCore Runtime. To do so we need to:

* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint decorator`
* Let AgentCoreRuntime control the running of the agent with `app.run()`

In [4]:
%%writefile weather_agent_strands.py
import argparse, json
from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

@tool
def weather(city: str):
    """
    Get weather for a given city
    Params:
        city: Name of the city you want the weather for
    """
    return f"Weather in {city} is warm and sunny"

agent = Agent(tools=[weather])

@app.entrypoint
def strands_agent_bedrock(payload):
    prompt = payload.get("prompt","hello")
    response = agent(prompt)
    return response

if __name__ == "__main__":
    app.run()

Overwriting weather_agent_strands.py


#### What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

#### 2. Create a role that Agent will use during run time
To run agent or tool in AgentCore Runtime you need an AWS Identity and Access Management execution role. The AgentCore Runtime execution role is an IAM role that AgentCore Runtime assumes to run an agent. 
Starting point for your role definition can be found at https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-permissions.html. In the role definition below we have added additional actions to support identity.

In [5]:
import boto3
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
sts = boto3.client('sts')
account_id = sts.get_caller_identity().get("Account")

In [6]:
permission = """{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "GetWorkloadAccessTokenForJWT",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetWorkloadAccessTokenForJWT"
            ],
            "Resource": "*"
        },
        {
            "Sid": "GetResourceOauth2Token",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceOauth2Token"
            ],
            "Resource": "*"
        },
        {
            "Sid": "GetWorkloadAccessTokenForUserId",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetWorkloadAccessTokenForUserId"
            ],
            "Resource": "*"
        },
        {
            "Sid": "GetResourceAPIKey",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceApiKey"
            ],
            "Resource": "*"
        },
        {
            "Sid": "SecretManager",
            "Effect": "Allow",
            "Action": [
                "secretsmanager:GetSecretValue"
            ],
            "Resource": "*"
        },
        {
            "Sid": "ECRImageAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:BatchGetImage",
                "ecr:GetDownloadUrlForLayer"
            ],
            "Resource": [
                "arn:aws:ecr:region:accountId:repository/*"
            ]        
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:DescribeLogStreams",
                "logs:CreateLogGroup"
            ],
            "Resource": [
                "arn:aws:logs:region:accountId:log-group:/aws/bedrock-agentcore/runtimes/*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:DescribeLogGroups"
            ],
            "Resource": [
                "arn:aws:logs:region:accountId:log-group:*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                "arn:aws:logs:region:accountId:log-group:/aws/bedrock-agentcore/runtimes/*:log-stream:*"
            ]
        },
        {
            "Sid": "ECRTokenAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:GetAuthorizationToken"
            ],
            "Resource": "*"
        },
        {
        "Effect": "Allow", 
        "Action": [ 
            "xray:PutTraceSegments", 
            "xray:PutTelemetryRecords", 
            "xray:GetSamplingRules", 
            "xray:GetSamplingTargets"
            ],
         "Resource": [ "*" ] 
         },
         {
            "Effect": "Allow",
            "Resource": "*",
            "Action": "cloudwatch:PutMetricData",
            "Condition": {
                "StringEquals": {
                    "cloudwatch:namespace": "bedrock-agentcore"
                }
            }
        },
        {
            "Sid": "GetAgentAccessToken",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetWorkloadAccessToken",
                "bedrock-agentcore:GetWorkloadAccessTokenForJWT",
                "bedrock-agentcore:GetWorkloadAccessTokenForUserId"
            ],
            "Resource": [
              "arn:aws:bedrock-agentcore:region:accountId:workload-identity-directory/default",
              "arn:aws:bedrock-agentcore:region:accountId:workload-identity-directory/default/workload-identity/agentName-*"
            ]
        },
         {"Sid": "BedrockModelInvocation", 
         "Effect": "Allow", 
         "Action": [ 
                "bedrock:InvokeModel", 
                "bedrock:InvokeModelWithResponseStream"
              ], 
        "Resource": [
            "arn:aws:bedrock:*::foundation-model/*",
            "arn:aws:bedrock:region:accountId:*"
        ]
        }
    ]
}"""

trust_policy = """{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "AssumeRolePolicy",
      "Effect": "Allow",
      "Principal": {
        "Service": "bedrock-agentcore.amazonaws.com"
      },
      "Action": "sts:AssumeRole",
      "Condition": {
            "StringEquals": {
                "aws:SourceAccount": "accountId"
            },
            "ArnLike": {
                "aws:SourceArn": "arn:aws:bedrock-agentcore:region:accountId:*"
            }
       }
    }
  ]
}"""
#trust_policy = json.loads(trust_policy.replace("accountId", account_id).replace("region", region))
trust_policy = trust_policy.replace("accountId", account_id).replace("region", region)
#permission = json.loads(permission.replace("accountId", account_id).replace("region", region))
permission = permission.replace("accountId", account_id).replace("region", region)

In [7]:
agentcore_role_name = "BedrockAgentCoreRole"

In [8]:
iam_client = boto3.client('iam')
agentcore_role_name = "BedrockAgentCoreRole"
policy_name = "BedrockAgentCorePolicy"
agentcore_iam_role = iam_client.create_role(
        RoleName=agentcore_role_name,
        AssumeRolePolicyDocument=trust_policy
    )

In [9]:
iam_client.put_role_policy(
        PolicyDocument=permission,
        PolicyName="AgentCorePolicy",
        RoleName=agentcore_role_name
    )

{'ResponseMetadata': {'RequestId': '83aff5c7-28af-4ef0-bba6-3d5fc6f1d2a2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 09 Aug 2025 18:15:10 GMT',
   'x-amzn-requestid': '83aff5c7-28af-4ef0-bba6-3d5fc6f1d2a2',
   'content-type': 'text/xml',
   'content-length': '206'},
  'RetryAttempts': 0}}

## Learning Objective 3: Deploy Agent into AgentCore
You can build an Agent Docker image and deploy it on AgentCore Runtime using 2 methods
* Use Bedrock AgentCore Runtime starter toolkit to simplify the process of Docker build, ECR creation, and deployment
* Retain control of the whole process by building a Docker Image, creating an ECR, and use boto3 SDK to deploy into Amazon Bedrock AgentCore

### Option 1 - Use AgentCore Starter Toolkit

#### Step 1: Configure Runtime
During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="./images/configure.png" width="60%"/>
</div>

In [10]:
%%writefile requirements.txt
strands-agents
strands-agents-tools
uv
boto3
bedrock-agentcore
bedrock-agentcore-starter-toolkit
tavily-python
uvicorn
fastapi

Writing requirements.txt


In [11]:
from bedrock_agentcore_starter_toolkit import Runtime
agentcore_runtime = Runtime()

agent_name = "weather_agent_strands"
config_response = agentcore_runtime.configure(
    entrypoint="weather_agent_strands.py",
    execution_role="BedrockAgentCoreRole",
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)

Bedrock AgentCore configured: /Users/dhegde/Documents/Code/AgentCore/GITHUB/amazon-bedrock-agentcore-samples/04-LearningPath/.bedrock_agentcore.yaml


#### Step 2: Launch Agent
Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="./images/launch.png" width="75%"/>
</div>

In [12]:
launch_response = agentcore_runtime.launch()

Build: #9 4.741 ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


The push refers to repository [135808924095.dkr.ecr.us-west-2.amazonaws.com/bedrock_agentcore-weather_agent_strands]
efc6cdef2f3b: Waiting
a4ad6ba921cf: Waiting
9d9293313685: Waiting
94fa42b5194c: Waiting
8671f17abb36: Waiting
f8d7a1f0426e: Waiting
33b4520294a0: Waiting
046e0ba021a5: Waiting
b3407f3b5b5b: Waiting
27b1542b9257: Waiting
9d9293313685: Waiting
efc6cdef2f3b: Waiting
a4ad6ba921cf: Waiting
b3407f3b5b5b: Waiting
27b1542b9257: Waiting
94fa42b5194c: Waiting
8671f17abb36: Waiting
f8d7a1f0426e: Waiting
33b4520294a0: Waiting
046e0ba021a5: Waiting
94fa42b5194c: Layer already exists
8671f17abb36: Waiting
f8d7a1f0426e: Layer already exists
33b4520294a0: Waiting
046e0ba021a5: Layer already exists
b3407f3b5b5b: Layer already exists
27b1542b9257: Waiting
efc6cdef2f3b: Waiting
a4ad6ba921cf: Waiting
9d9293313685: Waiting
a4ad6ba921cf: Waiting
27b1542b9257: Layer already exists
8671f17abb36: Waiting
33b4520294a0: Pushed
efc6cdef2f3b: Pushed
a4ad6ba921cf: Pushed
9d9293313685: Pushed
8671f17a

Deployed to cloud: arn:aws:bedrock-agentcore:us-west-2:135808924095:runtime/weather_agent_strands-p07lnvG3xt
🔍 Agent logs available at:
   /aws/bedrock-agentcore/runtimes/weather_agent_strands-p07lnvG3xt-DEFAULT
   /aws/bedrock-agentcore/runtimes/weather_agent_strands-p07lnvG3xt-DEFAULT/runtime-logs
💡 Tail logs with: aws logs tail /aws/bedrock-agentcore/runtimes/weather_agent_strands-p07lnvG3xt-DEFAULT --follow
💡 Or view recent logs: aws logs tail /aws/bedrock-agentcore/runtimes/weather_agent_strands-p07lnvG3xt-DEFAULT --since 1h


### Option 2 - Use Docker Container and Boto3 
AgentCore Starter toolkit abstracted the task of building a docker file and adding /invocation and /ping endpoints to your agent. However, you may have a need to build and deploy your agent in your own docker image. To align with Amazon Bedrock AgentCore requirements:
* Your agent must expose /invocations POST and /ping GET endpoints and be packaged in a Docker container
* Your docker images must be built for arm64

<div style="text-align:left">
    <img src="./images/launch_boto3.jpg" width="75%"/>
</div>

In [13]:
agent_name = "weather_agent_strands_wo_starter"

#### Step 1 - Update agent code
This implementation:
* Creates a FastAPI application with the required endpoints
* Initializes a Strands agent for processing user messages
* Implements the /invocations POST endpoint for agent interactions
* Implements the /ping GET endpoint for health checks
* Configures the server to run on host 0.0.0.0 and port 8080

In [14]:
%%writefile weather_agent_strands_wo_toolkit.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Dict, Any
from datetime import datetime
from strands import Agent, tool

app = FastAPI(title="Strands Agent Server", version="1.0.0")

@tool
def weather(city: str):
    """
    Get weather for a given city
    Params:
        city: Name of the city you want the weather for
    """
    return f"Weather in {city} is warm and sunny"
    
# Initialize Strands agent
strands_agent = Agent(tools=[weather])

class InvocationRequest(BaseModel):
    input: Dict[str, Any]

class InvocationResponse(BaseModel):
    output: Dict[str, Any]

    
@app.post("/invocations", response_model=InvocationResponse)
async def invoke_agent(request: InvocationRequest):
    try:
        user_message = request.input.get("prompt", "")
        if not user_message:
            raise HTTPException(
                status_code=400, 
                detail="No prompt found in input. Please provide a 'prompt' key in the input."
            )

        result = strands_agent(user_message)
        response = {
            "message": result.message,
            "timestamp": datetime.utcnow().isoformat(),
            "model": "strands-agent",
        }

        return InvocationResponse(output=response)

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Agent processing failed: {str(e)}")

@app.get("/ping")
async def ping():
    return {"status": "healthy"}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8080)

Writing weather_agent_strands_wo_toolkit.py


#### Step 2 - Build Docker container
* Application should be launched on 0.0.0.0 and port 8080 for AgentCore Runtime to be able to invoke it.

Detailed instructions can be found at https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

In [15]:
%%writefile Dockerfile
# Use uv's ARM64 Python base image
FROM --platform=linux/arm64 ghcr.io/astral-sh/uv:python3.11-bookworm-slim

WORKDIR /app

# Copy uv files
COPY requirements.txt .

# Install dependencies (including strands-agents)
#RUN pip install -r requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Copy agent file
COPY weather_agent_strands_wo_toolkit.py ./

# Expose port
EXPOSE 8080

# Run application
CMD ["uv", "run", "uvicorn", "weather_agent_strands_wo_toolkit:app", "--host", "0.0.0.0", "--port", "8080"]

Overwriting Dockerfile


In [16]:
!docker buildx create --use

musing_kilby


In [17]:
!docker buildx build --quiet --platform linux/arm64 -t weather-agent:arm64 --load .

sha256:9cd71e1b8b14715e5f0cef3137d38df23ac2c7e92919d60738b9e74e156f7a88


In [18]:
!aws ecr create-repository --repository-name {agent_name} --region {region}


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'weather_agent_strands_wo_starter' already exists in the registry with id '135808924095'


In [19]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.us-west-2.amazonaws.com

Login Succeeded


In [22]:
!docker buildx build --platform linux/arm64 -t {account_id}.dkr.ecr.us-west-2.amazonaws.com/{agent_name}:latest --push .

[+] Building 0.0s (0/0)         docker-container:musing_kilby
[+] Building 0.0s (0/1)                           docker-container:musing_kilby
[+] Building 0.2s (1/2)                           docker-container:musing_kilby
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 596B                                       0.0s
 => WARN: FromPlatformFlagConstDisallowed: FROM --platform flag should no  0.0s
 => [internal] load metadata for ghcr.io/astral-sh/uv:python3.11-bookworm  0.2s
[+] Building 0.3s (1/2)                           docker-container:musing_kilby
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 596B                                       0.0s
 => WARN: FromPlatformFlagConstDisallowed: FROM --platform flag should no  0.0s
 => [internal] load metadata for ghcr.io/astral-sh/uv:python3.11-bookworm  0.3s
[+] Building 0.4s (2/2)                           docker-c

#### Step 3 - Use Boto3 to create Agent on AgentCore Runtime

In [23]:
agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)

In [24]:
runtime_response = agentcore_control_client.create_agent_runtime(
    agentRuntimeName = agent_name,
    agentRuntimeArtifact={
        'containerConfiguration': {
            'containerUri': f'{account_id}.dkr.ecr.us-west-2.amazonaws.com/{agent_name}:latest'
        }
    },
    networkConfiguration={"networkMode": "PUBLIC"},
    roleArn=f'arn:aws:iam::{account_id}:role/{agentcore_role_name}'
)

In [25]:
runtime_response["agentRuntimeArn"]

'arn:aws:bedrock-agentcore:us-west-2:135808924095:runtime/weather_agent_strands_wo_starter-bguDoT3AvE'

## Learning Objective 4: Invoke Agent
<div style="text-align:left">
    <img src="./images/invoke.png" width="60%"/>
</div>

### Option 1 - Using toolkit
You can test the Agent that you have just deployed using the 'agentcore_runtime' object. Other two options (boto3 and HTTPS endpoint) are the options that you should use for production applications
* Here we are invoking the first agent built using starter toolkit

In [26]:
agentcore_runtime.invoke({"prompt": "How is the weather in Irvine?"})

{'ResponseMetadata': {'RequestId': '95898424-38ec-463f-bf61-316854bdee14',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 09 Aug 2025 18:18:53 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '95898424-38ec-463f-bf61-316854bdee14',
   'baggage': 'Self=1-68979109-524dac8748aed042255b347d,session.id=e7f5cb4a-02bc-479d-9cb8-c5d402ea24fe',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'e7f5cb4a-02bc-479d-9cb8-c5d402ea24fe',
   'x-amzn-trace-id': 'Root=1-68979109-349fe0325e184c042e311518;Self=1-68979109-524dac8748aed042255b347d'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'e7f5cb4a-02bc-479d-9cb8-c5d402ea24fe',
 'traceId': 'Root=1-68979109-349fe0325e184c042e311518;Self=1-68979109-524dac8748aed042255b347d',
 'baggage': 'Self=1-68979109-524dac8748aed042255b347d,session.id=e7f5cb4a-02bc-479d-9cb8-c5d402ea24fe',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': [b'"The weather

### Option 2 - Using boto3
* Invoking the second agent deployed using custom docker and agent code

In [27]:
agentcore_client = boto3.client("bedrock-agentcore", region_name = region)
payload = json.dumps({
    "input": {"prompt": "How is the weather in Irvine?"}
})
response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn = runtime_response["agentRuntimeArn"],
    qualifier="DEFAULT",
    payload=payload
)
response_body = response['response'].read()
response_data = json.loads(response_body)
print("Agent Response:", response_data)

Agent Response: {'output': {'message': {'role': 'assistant', 'content': [{'text': 'The weather in Irvine is warm and sunny! It sounds like a beautiful day there.'}]}, 'timestamp': '2025-08-09T18:19:09.808314', 'model': 'strands-agent'}}


#### Option 3 - Using https endpoint
* Invoking the second agent using `runtime_response["agentRuntimeArn"]` as rthe ARN.
* Can be used to invoke the first agent as well. Replace `runtime_response["agentRuntimeArn"]` with `launch_response.agent_arn`

In [28]:
import urllib.parse, requests

# URL encode the agent ARN
escaped_agent_arn = urllib.parse.quote(runtime_response["agentRuntimeArn"], safe='')

# Construct the URL
url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"
http_response = requests.post(url, data=json.dumps(payload))

In [29]:
http_response.status_code, http_response.text

(403, '{"message":"Missing Authentication Token"}')

#### Why did we get a HTTP 403 (Forbidden)?
While invoking an Agent using a toolkit and boto3 SDK, we are using the IAM role of the session to invoke the agenty. However, when you invoke it as a HTTPS endpoint, there is no authorization passed to the agent. Hence the 403 Forbidden error.

## Learning Objective 5 - Tools accessing protected resources
In this learning objective, we focus on authentication within the AgentCore ecosystem. In this objective we focus on authorization within an agent so that it can access protected resources. 
The credential management system supports multiple credential types including OAuth2 access tokens, API keys, client certificates, SAML assertions, and custom authentication tokens.

#### 1 - Tool authentication using API Key
* Setup an API key credential provider for Tavily
* Get the API key from credential provider at the start of the session

##### Create credential provider
* What they do: Act as smart middlemen that connect your agents to different services and identity systems
* Built-in providers: Come ready-to-use for popular services like:
  - Google
  - GitHub  
  - Slack
  - Salesforce
* Pre-configured: These providers already have the right settings and endpoints set up for you
* Custom options: You can create your own providers for any service that uses OAuth2 authentication
* Main benefit: Saves you time by handling the complex setup work automatically

In [32]:
import os
#os.environ["tavily_key"] = "tvly-dev-tavily-api-key"
from bedrock_agentcore.services.identity import IdentityClient
id_client = IdentityClient(region=region)
id_client.create_api_key_credential_provider({
    "name":"tavily_key_provider",
    "apiKey":os.environ["tavily_key"]
})

{'ResponseMetadata': {'RequestId': 'c640501c-c2d6-4c6a-9c9d-dcffa5677837',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 09 Aug 2025 18:20:23 GMT',
   'content-type': 'application/json',
   'content-length': '326',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c640501c-c2d6-4c6a-9c9d-dcffa5677837',
   'x-amzn-remapped-x-amzn-requestid': 'a0983c69-261f-4fec-9b9f-c2e16d199041',
   'x-amzn-remapped-content-length': '326',
   'x-amzn-remapped-connection': 'keep-alive',
   'x-amz-apigw-id': 'PDOoJGdsPHcEuBQ=',
   'x-amzn-trace-id': 'Root=1-68979166-17f98d8159b4cbbe2f86b186',
   'x-amzn-remapped-date': 'Sat, 09 Aug 2025 18:20:23 GMT'},
  'RetryAttempts': 0},
 'apiKeySecretArn': {'secretArn': 'arn:aws:secretsmanager:us-west-2:135808924095:secret:bedrock-agentcore-identity!default/apikey/tavily_key_provider-3yzalO'},
 'name': 'tavily_key_provider',
 'credentialProviderArn': 'arn:aws:bedrock-agentcore:us-west-2:135808924095:token-vault/default/apikeycredentialprovider/tavily

##### Use credential provider to get API key
* You have API keys in the AgentCore Identity vault
* Retrieve directly in your agent using the AgentCore SDK and the `@requires_api_key annotation`

In [33]:
%%writefile weather_agent_tavily.py
import argparse, json, os
from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.identity.auth import requires_api_key
from tavily import TavilyClient
import asyncio

TAVILY_API_KEY_FROM_CREDS_PROVIDER = ""

@requires_api_key(
    provider_name="tavily_key_provider", # replace with your own credential provider name
    into="api_key"
)
async def need_api_key(*, api_key: str):
    global TAVILY_API_KEY_FROM_CREDS_PROVIDER
    print(f'received api key for async func: {api_key}')
    TAVILY_API_KEY_FROM_CREDS_PROVIDER = api_key

app = BedrockAgentCoreApp()


@tool
def weather(city: str):
    """
    Get weather for a given city
    Params:
        city: Name of the city you want the weather for
    """
    tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
    response = tavily_client.search(f"What is current weather in {city}")
    return response
    
agent = Agent(tools=[weather])

@app.entrypoint
async def strands_agent_bedrock(payload):
    
    global TAVILY_API_KEY_FROM_CREDS_PROVIDER
    if not TAVILY_API_KEY_FROM_CREDS_PROVIDER:
        print("Attempting to retrieve API key...")
        try:
            await need_api_key(api_key="")
            print(f"API key retrieved: '{TAVILY_API_KEY_FROM_CREDS_PROVIDER}'")
            os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY_FROM_CREDS_PROVIDER
            print("Environment variable TAVILY_API_KEY set")
        except Exception as e:
            print(f"Error retrieving API key: {e}")
            raise
    else:
        print("API key already available")
        
    prompt = payload.get("prompt","hello")
    response = agent(prompt)
    return response

if __name__ == "__main__":
    app.run()

Writing weather_agent_tavily.py


In [34]:
agent_name = "weather_agent_tavily"
from bedrock_agentcore_starter_toolkit import Runtime
agentcore_runtime = Runtime()
config_response = agentcore_runtime.configure(
    entrypoint="weather_agent_tavily.py",
    execution_role="BedrockAgentCoreRole",
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
)

Bedrock AgentCore configured: /Users/dhegde/Documents/Code/AgentCore/GITHUB/amazon-bedrock-agentcore-samples/04-LearningPath/.bedrock_agentcore.yaml


In [35]:
tavily_agent_lanch = agentcore_runtime.launch()

Build: #9 4.425 ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


The push refers to repository [135808924095.dkr.ecr.us-west-2.amazonaws.com/bedrock_agentcore-weather_agent_tavily]
94fa42b5194c: Waiting
f8d7a1f0426e: Waiting
b30f3b6f638d: Waiting
28acd563392e: Waiting
27b1542b9257: Waiting
046e0ba021a5: Waiting
bd8295b8e16d: Waiting
8d15f357eba2: Waiting
67cd76f1c048: Waiting
b3407f3b5b5b: Waiting
28acd563392e: Waiting
27b1542b9257: Waiting
94fa42b5194c: Waiting
f8d7a1f0426e: Waiting
b30f3b6f638d: Waiting
67cd76f1c048: Waiting
b3407f3b5b5b: Waiting
046e0ba021a5: Waiting
bd8295b8e16d: Waiting
8d15f357eba2: Waiting
bd8295b8e16d: Waiting
8d15f357eba2: Waiting
67cd76f1c048: Waiting
b3407f3b5b5b: Layer already exists
046e0ba021a5: Layer already exists
f8d7a1f0426e: Layer already exists
b30f3b6f638d: Waiting
28acd563392e: Waiting
27b1542b9257: Layer already exists
94fa42b5194c: Layer already exists
28acd563392e: Pushed
67cd76f1c048: Pushed
bd8295b8e16d: Pushed
8d15f357eba2: Pushed
b30f3b6f638d: Pushed
latest: digest: sha256:f2d92117e2715c7d63f8a791d29cf5b

Deployed to cloud: arn:aws:bedrock-agentcore:us-west-2:135808924095:runtime/weather_agent_tavily-fnxYNB6m93
🔍 Agent logs available at:
   /aws/bedrock-agentcore/runtimes/weather_agent_tavily-fnxYNB6m93-DEFAULT
   /aws/bedrock-agentcore/runtimes/weather_agent_tavily-fnxYNB6m93-DEFAULT/runtime-logs
💡 Tail logs with: aws logs tail /aws/bedrock-agentcore/runtimes/weather_agent_tavily-fnxYNB6m93-DEFAULT --follow
💡 Or view recent logs: aws logs tail /aws/bedrock-agentcore/runtimes/weather_agent_tavily-fnxYNB6m93-DEFAULT --since 1h


In [37]:
#agentcore_runtime.invoke(payload={"prompt": "How is the weather in Irvine?"})
agentcore_runtime.invoke(payload={"prompt": "How is the weather in Irvine?"}, user_id=sts.get_caller_identity()["UserId"])

{'ResponseMetadata': {'RequestId': 'cb1690de-de49-4a0b-af11-34bca1586610',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 09 Aug 2025 18:22:32 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'cb1690de-de49-4a0b-af11-34bca1586610',
   'baggage': 'Self=1-689791e0-561f0cc7391f087929645f86,session.id=e580ea69-94ac-41a8-911f-322c3717d46d',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'e580ea69-94ac-41a8-911f-322c3717d46d',
   'x-amzn-trace-id': 'Root=1-689791e0-7dda6a3a7cc9cb144c0706d1;Self=1-689791e0-561f0cc7391f087929645f86'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'e580ea69-94ac-41a8-911f-322c3717d46d',
 'traceId': 'Root=1-689791e0-7dda6a3a7cc9cb144c0706d1;Self=1-689791e0-561f0cc7391f087929645f86',
 'baggage': 'Self=1-689791e0-561f0cc7391f087929645f86,session.id=e580ea69-94ac-41a8-911f-322c3717d46d',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': [b'"The current

#### 2 - Tools needing OAuth Token - M2M Client Credentials

##### AgentCore Identity offers two ways for agents to authenticate
* User-delegated access (OAuth 2.0 authorization code grant):
  - Agent acts on behalf of a specific user
  - User gives permission for the agent to access their data
  - Good for personal assistants or user-specific tasks
* Machine-to-machine authentication (OAuth 2.0 client credentials grant):
  - Agent acts independently without a specific user
  - Direct service-to-service communication
  - Good for automated systems or background processes

In this leanring objective we will explore M2M authentication.

In [39]:
from m2m_cognito_setup import create_m2m_pool
cognito_config = create_m2m_pool()

Creating User Pool...
✅ User Pool created with ID: us-west-2_98I0xNVXN
Creating Resource Server with custom scopes...
✅ Resource Server created with ID: https://api.myapp.com
Creating M2M App Client...
✅ M2M Client created with ID: 2slvfnl7u6mc5cfpk9e3ngrh17
Creating User Pool Domain: m2m-domain-63846
✅ Domain created: m2m-domain-63846
Creating test user...
✅ Test user created: m2m-test-user / TestPass123!

Machine-to-Machine Client Created Successfully!
User Pool ID: us-west-2_98I0xNVXN
Resource Server ID: https://api.myapp.com
Client ID: 2slvfnl7u6mc5cfpk9e3ngrh17
Credentials saved to: m2m_credentials.json


AgentCore Identity provice built in provider clients for Google. GitHub, Microsoft, Salesforce, and Slack. It also allows you to setup custom providers. 
In this example, we will use a custom provider that uses Amazon Cognito M2M Pool

In [40]:
from bedrock_agentcore.services.identity import IdentityClient
        
identity_client = IdentityClient(region="us-west-2")

#Configure Cognito OAuth2 Provider - 2LO Example
cognito_provider = identity_client.create_oauth2_credential_provider({
  "name": "cognito-provider-m2m-pool-agentcore",
  "credentialProviderVendor": "CustomOauth2",
  "oauth2ProviderConfigInput": {
    "customOauth2ProviderConfig": {
      "clientId": cognito_config["client_id"],
      "clientSecret": cognito_config["client_secret"],
      "oauthDiscovery": {
        'authorizationServerMetadata': {
            'issuer': "https://cognito-idp.us-west-2.amazonaws.com/us-west-2_ZGLlSyvTI",
            'authorizationEndpoint': f"https://{cognito_config['domain_name']}.auth.us-west-2.amazoncognito.com/oauth2/authorize",
            'tokenEndpoint': f"https://{cognito_config['domain_name']}.auth.us-west-2.amazoncognito.com/oauth2/token",
            'responseTypes': ["code","token"],
        }
      }
    }
  }
})
print(cognito_provider)

{'ResponseMetadata': {'RequestId': '33cb6a33-5c7c-4ec3-b8e5-65fe2a3c8bff', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 09 Aug 2025 18:24:24 GMT', 'content-type': 'application/json', 'content-length': '825', 'connection': 'keep-alive', 'x-amzn-requestid': '33cb6a33-5c7c-4ec3-b8e5-65fe2a3c8bff', 'x-amzn-remapped-x-amzn-requestid': 'c8e5962d-fd00-4224-889a-c06e804e12c5', 'x-amzn-remapped-content-length': '825', 'x-amzn-remapped-connection': 'keep-alive', 'x-amz-apigw-id': 'PDPN7GqFvHcEoGA=', 'x-amzn-trace-id': 'Root=1-68979258-6247541f4d4d7e254d3d6ef9', 'x-amzn-remapped-date': 'Sat, 09 Aug 2025 18:24:24 GMT'}, 'RetryAttempts': 0}, 'clientSecretArn': {'secretArn': 'arn:aws:secretsmanager:us-west-2:135808924095:secret:bedrock-agentcore-identity!default/oauth2/cognito-provider-m2m-pool-agentcore-R8ZDuH'}, 'name': 'cognito-provider-m2m-pool-agentcore', 'credentialProviderArn': 'arn:aws:bedrock-agentcore:us-west-2:135808924095:token-vault/default/oauth2credentialprovider/cognito-provi

In [41]:
%%writefile weather_agent_cognito.py
import argparse, json, os
from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.identity.auth import requires_access_token
from tavily import TavilyClient
import asyncio

COGNITO_ACESS_TOKEN = ""

@requires_access_token(
    provider_name="cognito-provider-m2m-pool-agentcore", # replace with your own credential provider name
    into="access_token",
    auth_flow="M2M",
    scopes = ["https://api.myapp.com/read"],
    force_authentication=True
)
async def need_access_token(*, access_token: str):
    print("")
    global COGNITO_ACESS_TOKEN
    print(f'received acess token for async func: {access_token}')
    COGNITO_ACESS_TOKEN = access_token

app = BedrockAgentCoreApp()

@tool
def weather(city: str):
    """
    Get weather for a given city
    Params:
        city: Name of the city you want the weather for
    """
    if COGNITO_ACESS_TOKEN:
        return f"Weather in {city} is warm and sunny"
    else:
        return "Cannot access weather API. Access Token missing"
    
    
agent = Agent(tools=[weather])

@app.entrypoint
async def strands_agent_bedrock(payload):
    
    global COGNITO_ACESS_TOKEN
    if not COGNITO_ACESS_TOKEN:
        print("Attempting to retrieve Token...")
        try:
            await need_access_token(access_token="")
            print(f"API key retrieved: '{COGNITO_ACESS_TOKEN}'")
            os.environ["COGNITO_ACESS_TOKEN"] = COGNITO_ACESS_TOKEN
            print("Environment variable COGNITO_ACESS_TOKEN set")
        except Exception as e:
            print(f"Error retrieving API key: {e}")
            raise
    else:
        print("Access token already available")
        
    prompt = payload.get("prompt","hello")
    response = agent(prompt)
    return response

if __name__ == "__main__":
    app.run()

Writing weather_agent_cognito.py


In [42]:
!rm /Users/dhegde/Documents/Code/AgentCore/.bedrock_agentcore.yaml

In [43]:
agent_name = "weather_agent_cognito"
from bedrock_agentcore_starter_toolkit import Runtime
agentcore_runtime = Runtime()
config_response = agentcore_runtime.configure(
    entrypoint="weather_agent_cognito.py",
    execution_role="BedrockAgentCoreRole",
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
)

Bedrock AgentCore configured: /Users/dhegde/Documents/Code/AgentCore/GITHUB/amazon-bedrock-agentcore-samples/04-LearningPath/.bedrock_agentcore.yaml


In [44]:
weather_agent_cognito_launch = agentcore_runtime.launch()

Build: #9 4.401 ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


The push refers to repository [135808924095.dkr.ecr.us-west-2.amazonaws.com/bedrock_agentcore-weather_agent_cognito]
806abc05d852: Waiting
f013ec0c87ae: Waiting
046e0ba021a5: Waiting
709687e0be6e: Waiting
efec7b901ca6: Waiting
bb92a866b42c: Waiting
b3407f3b5b5b: Waiting
94fa42b5194c: Waiting
27b1542b9257: Waiting
f8d7a1f0426e: Waiting
806abc05d852: Waiting
f013ec0c87ae: Waiting
046e0ba021a5: Waiting
709687e0be6e: Waiting
f8d7a1f0426e: Waiting
efec7b901ca6: Waiting
bb92a866b42c: Waiting
b3407f3b5b5b: Waiting
94fa42b5194c: Waiting
27b1542b9257: Waiting
f8d7a1f0426e: Layer already exists
efec7b901ca6: Waiting
bb92a866b42c: Waiting
b3407f3b5b5b: Layer already exists
94fa42b5194c: Layer already exists
27b1542b9257: Layer already exists
806abc05d852: Waiting
f013ec0c87ae: Waiting
046e0ba021a5: Layer already exists
709687e0be6e: Waiting
806abc05d852: Pushed
f013ec0c87ae: Pushed
efec7b901ca6: Pushed
bb92a866b42c: Pushed
709687e0be6e: Pushed
latest: digest: sha256:c29a24a68cbebe1a978d4a86749c34

Deployed to cloud: arn:aws:bedrock-agentcore:us-west-2:135808924095:runtime/weather_agent_cognito-r75qsDCV2y
🔍 Agent logs available at:
   /aws/bedrock-agentcore/runtimes/weather_agent_cognito-r75qsDCV2y-DEFAULT
   /aws/bedrock-agentcore/runtimes/weather_agent_cognito-r75qsDCV2y-DEFAULT/runtime-logs
💡 Tail logs with: aws logs tail /aws/bedrock-agentcore/runtimes/weather_agent_cognito-r75qsDCV2y-DEFAULT --follow
💡 Or view recent logs: aws logs tail /aws/bedrock-agentcore/runtimes/weather_agent_cognito-r75qsDCV2y-DEFAULT --since 1h


In [47]:
agentcore_runtime.invoke(payload={"prompt": "How is the weather in Irvine?"}, user_id=sts.get_caller_identity()["UserId"])

{'ResponseMetadata': {'RequestId': 'a3ab1f58-7abc-4991-92eb-079ca30d142e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 09 Aug 2025 18:26:04 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a3ab1f58-7abc-4991-92eb-079ca30d142e',
   'baggage': 'Self=1-689792b8-0c620bd53504bd5f39ab5f82,session.id=204b0041-aab1-4a71-b31a-ff6efb326a0e',
   'x-amzn-bedrock-agentcore-runtime-session-id': '204b0041-aab1-4a71-b31a-ff6efb326a0e',
   'x-amzn-trace-id': 'Root=1-689792b8-43cb9ee346b0f6af1124a0d8;Self=1-689792b8-0c620bd53504bd5f39ab5f82'},
  'RetryAttempts': 0},
 'runtimeSessionId': '204b0041-aab1-4a71-b31a-ff6efb326a0e',
 'traceId': 'Root=1-689792b8-43cb9ee346b0f6af1124a0d8;Self=1-689792b8-0c620bd53504bd5f39ab5f82',
 'baggage': 'Self=1-689792b8-0c620bd53504bd5f39ab5f82,session.id=204b0041-aab1-4a71-b31a-ff6efb326a0e',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': [b'"The weather

## Learning Objective 6 - Allowing only authorized users from invoking an Agent
Two authentication options for hosted agents:

IAM SigV4 Authentication (Default):
* Works automatically without setup
* Same as other AWS APIs

JWT Bearer Token Authentication (Optional):
* Requires configuration during agent creation
* Need to provide two settings:

  - **Discovery URL**: Must end with /.well-known/openid-configuration
  - **Allowed clients**: List of permitted client IDs (validates client_id claim)

* When to use each:
  - Use IAM SigV4 for standard AWS integration
  - Use JWT for external identity providers or custom auth systems

In [48]:
#import importlib
#importlib.reload(user_cognito_setup)

#### Step 1: Create a cognito pool for user authorization

In [50]:
import user_cognito_setup
spa_config = user_cognito_setup.create_spa_pool()

Using AWS Region: us-west-2

Created User Pool: DemoUserPool (ID: us-west-2_gitmOvIHk)
Created App Client: DemoClient (ID: 3ru1369kngk2253avn7ihoqu2a)

Creating testuser1...
Created user: testuser1
Successfully authenticated user: testuser1

Creating testuser2...
Created user: testuser2
Successfully authenticated user: testuser2


In [51]:
!rm /Users/dhegde/Documents/Code/AgentCore/.bedrock_agentcore.yaml

rm: /Users/dhegde/Documents/Code/AgentCore/.bedrock_agentcore.yaml: No such file or directory


#### Step 2: Add authorization configuration while creating Agent Runtime

In [58]:
#Update the Cognito Discovery url below. You can get the discovery url from the "Provision a Cognito User Pool" section
discovery_url = f'https://cognito-idp.us-west-2.amazonaws.com/{spa_config["pool_id"]}/.well-known/openid-configuration' 
client_id = spa_config['client_id']
inbound_auth_weather_rt = Runtime()
agent_name = "inbound_auth_weather"

response = inbound_auth_weather_rt.configure(
    entrypoint="weather_agent_strands.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id]
        }
    }
)

Bedrock AgentCore configured: /Users/dhegde/Documents/Code/AgentCore/GITHUB/amazon-bedrock-agentcore-samples/04-LearningPath/.bedrock_agentcore.yaml


In [59]:
inbound_auth_weather_launch = inbound_auth_weather_rt.launch()

Build: #9 4.479 ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


The push refers to repository [135808924095.dkr.ecr.us-west-2.amazonaws.com/bedrock_agentcore-inbound_auth_weather]
b3407f3b5b5b: Waiting
27b1542b9257: Waiting
94fa42b5194c: Waiting
4684bda3cff9: Waiting
9693a807dce6: Waiting
f8d7a1f0426e: Waiting
7edc634fb110: Waiting
38cebf204f8e: Waiting
046e0ba021a5: Waiting
5da544f60aed: Waiting
7edc634fb110: Waiting
38cebf204f8e: Waiting
046e0ba021a5: Waiting
5da544f60aed: Waiting
9693a807dce6: Waiting
f8d7a1f0426e: Waiting
94fa42b5194c: Waiting
4684bda3cff9: Waiting
b3407f3b5b5b: Waiting
27b1542b9257: Waiting
5da544f60aed: Waiting
9693a807dce6: Waiting
f8d7a1f0426e: Waiting
7edc634fb110: Waiting
38cebf204f8e: Waiting
046e0ba021a5: Waiting
4684bda3cff9: Waiting
b3407f3b5b5b: Waiting
27b1542b9257: Layer already exists
94fa42b5194c: Layer already exists
046e0ba021a5: Layer already exists
f8d7a1f0426e: Layer already exists
4684bda3cff9: Waiting
b3407f3b5b5b: Layer already exists
5da544f60aed: Pushed
4684bda3cff9: Pushed
9693a807dce6: Pushed
38cebf20

Deployed to cloud: arn:aws:bedrock-agentcore:us-west-2:135808924095:runtime/inbound_auth_weather-vjfiSF7G4u
🔍 Agent logs available at:
   /aws/bedrock-agentcore/runtimes/inbound_auth_weather-vjfiSF7G4u-DEFAULT
   /aws/bedrock-agentcore/runtimes/inbound_auth_weather-vjfiSF7G4u-DEFAULT/runtime-logs
💡 Tail logs with: aws logs tail /aws/bedrock-agentcore/runtimes/inbound_auth_weather-vjfiSF7G4u-DEFAULT --follow
💡 Or view recent logs: aws logs tail /aws/bedrock-agentcore/runtimes/inbound_auth_weather-vjfiSF7G4u-DEFAULT --since 1h


In [60]:
cognito_client = boto3.client('cognito-idp', region_name=region,)
cognito_bearer_token_1 = cognito_client.initiate_auth(
        ClientId=client_id,  
        AuthFlow="USER_PASSWORD_AUTH",
        AuthParameters={
            "USERNAME": "testuser1", 
            "PASSWORD": spa_config['user1']['password']
        }
    )["AuthenticationResult"]["AccessToken"]

In [61]:
import urllib.parse, requests

# URL encode the agent ARN
escaped_agent_arn = urllib.parse.quote(inbound_auth_weather_launch.agent_arn, safe='')

headers = {
    "Authorization": f"Bearer {cognito_bearer_token_1}",
    "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": "aea8996f-dcf5-4227-b5ea-f9e9c1843729",
    "X-Amzn-Trace-Id": "1234567890" 
}

# Construct the URL
url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"
http_response = requests.post(url, data=json.dumps({"prompt": "How is the weather in Irvine?"}), headers=headers)

In [62]:
http_response.text

'"The weather in Irvine is warm and sunny today! It sounds like a great day to be outside.\\n"'

In [57]:
!cat weather_agent.py

import argparse, json
from strands import Agent, tool

@tool
def weather(city: str):
    """
    Get weather for a given city
    Params:
        city: Name of the city you want the weather for
    """
    return f"Weather in {city} is warm and sunny"

agent = Agent(tools=[weather])

def strands_agent_bedrock(payload):
    prompt = payload.get("prompt","hello")
    response = agent(prompt)
    return response

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_bedrock(json.loads(args.payload))
    print(response)
